In [613]:
import numpy as np
import dill
from lkf_tools.dataset import *

In [614]:
lkf_data = dill.load(open("data/lkf_pickle_files/lkf_data_2015_005.pickle", "rb"))

In [615]:
lkfs_all = []
for it in lkf_data.indexes:
    lkfs_path = lkf_data.lkfpath.joinpath('lkf_%s_%03i.npy'
                                          % (lkf_data.netcdf_file.split('/')[-1].split('.')[0],
                                             (it+1))
                                         )
    lkfs_all.append(np.load(lkfs_path, allow_pickle=True))

In [616]:
print('day   number of lkfs')
for ind, item in enumerate(lkfs_all):
    print(ind+1, '   ', np.shape(item)[0])

day   number of lkfs
1     90
2     63
3     54
4     68
5     90
6     79
7     71
8     73
9     77


In [617]:
tracks_all = []

for it in lkf_data.indexes:
    tracks_path = lkf_data.track_output_path.joinpath('lkf_tracked_pairs_%s_to_%s.npy'
                                                      % (lkf_data.lkf_filelist[it][4:-4],
                                                         lkf_data.lkf_filelist[it+1][4:-4])
                                                     )
    tracks_all.append(np.load(tracks_path, allow_pickle=True))

In [618]:
##### here only one of the paths is saved if the lkf has
# two associated features in the following record #####

# lkf_tracks[i] are the lkf tracks that start at day i+1
lkf_tracks = np.zeros(len(lkfs_all),dtype='object')

# number all the lkfs on day one
for startday in range(len(lkfs_all)):
    lkf_tracks[startday] = np.arange(np.shape(lkfs_all[startday])[0], dtype='object')

    # get the lkf number on day two (if the lkf can be tracked)
    # and add it to the lkf
    for ind, item in enumerate(lkf_tracks[startday]):
        in_next_record = (item == tracks_all[startday][:,0])

        if np.any(in_next_record):
            pos_in_next_record = np.where(in_next_record)
            lkf_number_in_next_record = tracks_all[startday][pos_in_next_record,1]

            lkf_tracks[startday][ind] = np.append(item, lkf_number_in_next_record[0,0])

    # loop over the following days
    for i in range(1,len(tracks_all)-startday):
        for ind, item in enumerate(lkf_tracks[startday]):
            if np.array(item).size == i+1:
                in_next_record = (item[-1] == tracks_all[i+startday][:,0])

                if np.any(in_next_record):
                    pos_in_next_record = np.where(in_next_record)

                    lkf_number_in_next_record = tracks_all[i+startday][pos_in_next_record,1]

                    lkf_tracks[startday][ind] = np.append(item, lkf_number_in_next_record[0,0])

In [619]:
already_tracked = np.zeros(len(lkf_tracks)-1,dtype='object')
for it in range(len(lkf_tracks)-1):
    
    for ind, item in enumerate(lkf_tracks[it]):

        if np.array(item).size > 1:
            already_tracked[it] = [np.array(item).flat[1] if already_tracked[it]==0 else np.append(already_tracked[it],np.array(item).flat[1])]

for i in range(1,len(tracks_all)):
    lkf_tracks[i] = np.delete(lkf_tracks[i], already_tracked[i-1])

In [620]:
lifetimes = np.zeros_like(lkf_tracks, dtype='object')
for i in range(len(lkf_tracks)):
    lifetimes[i] = np.zeros_like(lkf_tracks[i], dtype='object')
    for ind, item in enumerate(lkf_tracks[i]):
        lifetimes[i][ind] = np.array(item).size

In [621]:
total_mean_lifetime = np.concatenate(lifetimes).mean()
total_mean_lifetime

1.7582938388625593

In [622]:
lifetimes_tracked_lkfs = [lifetimes[i][np.where(lifetimes[i]!=1)] for i in range(len(lifetimes))]
total_mean_lifetime_tracked_lkfs = np.concatenate(lifetimes_tracked_lkfs).mean()
total_mean_lifetime_tracked_lkfs

2.8181818181818183